# Final Project 2 - NoSQL Database for AirBnB

In [13]:
import pandas as pd
import json
from bson import json_util
from pymongo import MongoClient
client = MongoClient()
db = client.airbnb

In [21]:
#. What are people saying about me (somebody)?
#Ans. Fetch the Reviews for each of the matching Listing_id between Reviews and Property_Listings Collections.

cursor = db.Reviews.aggregate([ { "$lookup" :  {  "from": "Property_listings",  "localField": "listing_id",  "foreignField": "listing_id",  "as":"match_reviews"   } }])
i=0
for review in cursor:
    if i > 5:
        break
    else:
        print(review['listing_id'], review['comments'])
        print("----------")
        i+=1
    

95740 The host canceled this reservation 54 days before arrival. This is an automated posting.
----------
65419 The reservation was canceled 10 days before arrival. This is an automated posting.
----------

Wir haben uns rundum wohl gefühlt, vielen Dank dafür!
----------
65419 The host canceled this reservation 60 days before arrival. This is an automated posting.
----------
100407 Awesome host! Quick and easy communication. The apartment is as advertised. Clean and comfortable with easy access to the city. Residential neighborhood but close to some nightlife and tourist sights (museums, etc.). 

Had a great time in Abby's apartment would highly recommend it to anyone visiting Amsterdam and I'll definitely be staying there again when I come back! Thanks, Abby!
----------
83552 Great location and lovely apartment. Emma's sister met us at the apartment and got my wife and I situated. Would stay again without a seconds hesitation.
----------


In [23]:
#xi. Should I add and picture or url to my post?
#Ans. Check for likes/retweets of posts with and without URL.
#     IF count is of posts' likes/retweets with URL > count of posts' likes/retweets without URL
#     THEN
#     URL is should be metioned in post
#     ELSE
#     No need to add URL to post. 
c1 = db.TweetDetils.aggregate([ {"$match":{"urls":{"$size":0}}}, {"$group" : {"_id":"null", "score":{"$sum":"$retweet_cnt"}}},  {"$sort":{"score" : -1}}])
cnt_url=0
cnt_no_url = 0
for rec in c1:
    cnt_url = rec["score"]

c2 = db.TweetDetils.aggregate([ {"$match":{"urls":{"$size":1}}}, {"$group" : {"_id":"null", "score":{"$sum":"$retweet_cnt"}}},  {"$sort":{"score" : -1}}])
for rec in c2:
    cnt_no_url = rec["score"]
    
if cnt_url >= cnt_no_url:
    print("Should Contain URL")
else:
    print("Should not contain url")


Should not contain url


In [25]:
#ii. How viral are my posts?
#Ans. Get the sum of all the fav_cnt and group by users order by fav_cnt desc;
#     The user with most counts has the most viral posts.
cursor = db.TweetDetils.aggregate([ {"$group" : {"_id":"$user_name", "score":{"$sum":"$fav_cnt"}}},  {"$sort":{"score" : -1}},{"$limit":5}])
for rec in cursor:
    print(rec["_id"], rec["score"])

johnlegend 1209
CGTNOfficial 251
cottage_green 139
Miteymiss 122
Airbnb 122


In [26]:
#iii. How much influence to my posts have?
#Ans. Get the sum of all the retweeted tweets and group by users  
#     The users with most counts is most influencial
cursor = db.TweetDetils.aggregate([ {"$group" : {"_id":"$user_name", "score":{"$sum":"$retweet_cnt"}}},  {"$sort":{"score" : -1}},{"$limit":5}])
for rec in cursor:
    print(rec["_id"], rec["score"])

PAKO0699 13682
arianaisquaking 6841
Theus_Antonny 6841
intoyougabi 6841
theesztikocsis 6841
